In [17]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam


# Load the dataset
load_dir = './saved_dataset'
loaded_dataset = tf.data.experimental.load(load_dir)

batch_size = 32
loaded_dataset = loaded_dataset.batch(batch_size).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)

number_of_labels = 9


In [18]:
loaded_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300, 1024), dtype=tf.float32, name=None), TensorSpec(shape=(None, 9), dtype=tf.int32, name=None))>

In [19]:
model = Sequential([
    SimpleRNN(64, return_sequences=True),
    SimpleRNN(64),
    Dense(128, activation='relu'),
    Dense(number_of_labels, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(loaded_dataset, epochs=10)


Epoch 1/10
70/70 [==============================] - 9s 74ms/step - loss: 0.3727 - accuracy: 0.2665
Epoch 2/10
70/70 [==============================] - 8s 76ms/step - loss: 0.3363 - accuracy: 0.2920
Epoch 3/10
70/70 [==============================] - 8s 79ms/step - loss: 0.3359 - accuracy: 0.2951
Epoch 4/10
70/70 [==============================] - 9s 90ms/step - loss: 0.3353 - accuracy: 0.2915
Epoch 5/10
70/70 [==============================] - 8s 78ms/step - loss: 0.3358 - accuracy: 0.2920
Epoch 6/10
70/70 [==============================] - 9s 92ms/step - loss: 0.3348 - accuracy: 0.2987
Epoch 7/10
70/70 [==============================] - 9s 88ms/step - loss: 0.3355 - accuracy: 0.2996
Epoch 8/10
70/70 [==============================] - 9s 84ms/step - loss: 0.3348 - accuracy: 0.2853
Epoch 9/10
70/70 [==============================] - 9s 88ms/step - loss: 0.3355 - accuracy: 0.2888
Epoch 10/10
70/70 [==============================] - 10s 96ms/step - loss: 0.3347 - accuracy: 0.2991
